# BERT - Out of the Box

In this notebook, we will test the performance of an out-of-the-box BERT model on CommonsenseQA. I follow the tutorial here: https://github.com/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb

I've implemented the Hugginface transformers library. 

I referred to the Commonsense QA repo and code to understand how the authors of this work establiahsed their baseline using BERT. This is the link to their code: https://github.com/jonathanherzig/commonsenseqa/blob/master/bert/run_commonsense_qa.py

From this repo (README): https://github.com/jonathanherzig/commonsenseqa

Their work is far more advanced and complicated than maybe what I want to do at this time. But I refer to their work to understand the set up.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import warnings

import json
from pandas.io.json import json_normalize
warnings.filterwarnings('ignore')


from transformers import BertTokenizer, BertModel, BertConfig

import torch
from tensorflow import keras

from datetime import datetime

import time 
configuration = BertConfig() 
from IPython.display import Image 

from tensorflow.keras.callbacks import TensorBoard
import pickle

runtype="tiny"

print("Run type:", runtype)


Run type: tiny


## Import dataset

It's in the dataset folder.

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# %cd /content/drive/My Drive/NLP/w266-commonsenseqa/BERT_oob

/content/drive/My Drive/MIDS/NLP/w266-commonsenseqa/BERT_oob


In [4]:
def load_data(file):
    lines = []
    with open(file, 'rb') as json_file:
        for json_line in json_file:
            lines.append(json.loads(json_line))
        data = json_normalize(lines)
        data.columns = data.columns.map(lambda x: x.split(".")[-1])
    return data
# os.chdir('w266-commonsenseqa/BERT_oob)
train = load_data('../dataset/train_rand_split.jsonl')
dev = load_data('../dataset/dev_rand_split.jsonl')
train.head()

,answerKey,id,question_concept,choices,stem
0,A,075e483d21c29a511267ef62bedc0461,punishing,"[{'label': 'A', 'text': 'ignore'}, {'label': '...",The sanctions against the school were a punish...
1,B,61fe6e879ff18686d7552425a36344c8,people,"[{'label': 'A', 'text': 'race track'}, {'label...",Sammy wanted to go to where the people were. ...
2,A,4c1cb0e95b99f72d55c068ba0255c54d,choker,"[{'label': 'A', 'text': 'jewelry store'}, {'la...",To locate a choker not located in a jewelry bo...
3,D,02e821a3e53cb320790950aab4489e85,highway,"[{'label': 'A', 'text': 'united states'}, {'la...",Google Maps and other highway and street GPS s...
4,C,23505889b94e880c3e89cff4ba119860,fox,"[{'label': 'A', 'text': 'pretty flowers.'}, {'...","The fox walked from the city into the forest, ..."


## Steps

1. Import training examples
2. Process it
    - Format input into something BERT can work with, including `[CLS]` and `[SEP]`
    - We were thinking which label is correct: 
    - Tokenize 
    - Create an output layer using softmax. 
3. Train it
    - Specify how many layers of BERT to fine tune
    

# BERT base model (uncased)

From: https://huggingface.co/bert-base-uncased

> Pretrained model on English language using a masked language modeling (MLM) objective. It was introduced in this paper and first released in this repository. This model is uncased: it does not make a difference between english and English.
> 
> Disclaimer: The team releasing BERT did not write a model card for this model so this model card has been written by the Hugging Face team.

For each question, there are five answer choices. Only one of them is correct.

For BERT, the first thought was to have all five answers attached to each question, and the model would choose one of the five responses. This is how it's originally done in the CommonsenseQA paper.

```
[CLS] Question text here [SEP] Ans choice A [SEP] Ans choice B [SEP] Ans choice C [SEP] Ans choice D [SEP] Ans choice E [SEP]
```

It seems complicated, however, and requires a significant lift. So for now, let me try creating five question-answer pairs for each question. Like this:

```
[CLS] Question text here [SEP] Ans choice A [SEP]
[CLS] Question text here [SEP] Ans choice B [SEP]
[CLS] Question text here [SEP] Ans choice C [SEP]
[CLS] Question text here [SEP] Ans choice D [SEP]
[CLS] Question text here [SEP] Ans choice E [SEP]
```

Only one of the above 5 inputs will have a positive label for being the correct answer. The rest will have 0. The problem with this model is that we're evaluating each choice separately to see if it looks like a right answer at all. But I think it's important for the model to know how the answer choices compare to each other as well.


In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
lab_order = {"A": 0, "B":1, "C":2, "D":3, "E":4}

class InputExample(object):
    """A single multiple choice question and its five multiple choice answer candidates"""
    # This class is adapted from https://github.com/jonathanherzig/commonsenseqa/blob/master/bert/run_commonsense_qa.py
    # and from https://github.com/rodgzilla/pytorch-pretrained-BERT/blob/dcb50eaa4b80d3ab75d373c36780c80fb47cfd97/examples/run_swag.py

    def __init__(
            self,
            qid,
            question,
            choice_0,
            choice_1,
            choice_2,
            choice_3,
            choice_4,
            label=None):
        """Construct an instance."""
        self.qid = qid
        self.question = question  # e.g., 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?'
        self.choices = [          # All five anser choices as a list
            choice_0,
            choice_1,
            choice_2,
            choice_3,
            choice_4
        ]
        self.label = label        # 
        
    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        l = [
            f"qid: {self.qid}",
            f"question: {self.question}",
            f"choice_0: {self.choices[0]}",
            f"choice_1: {self.choices[1]}",
            f"choice_2: {self.choices[2]}",
            f"choice_3: {self.choices[3]}",
            f"choice_4: {self.choices[4]}",
        ]

        if self.label is not None:
            l.append(f"label: {self.label}")

        return ", ".join(l)    

class InputFeatures(object):
    """Adapted from: https://github.com/rodgzilla/pytorch-pretrained-BERT/blob/dcb50eaa4b80d3ab75d373c36780c80fb47cfd97/examples/run_swag.py
    Stores Bert model inputs (ids, masks) for each example"""
    
    def __init__(self,
                 example_id,
                 choices_features,
                 label

    ):
        self.example_id = example_id
        self.choices_features = [
            {
                'input_ids': input_ids,
                'input_mask': input_mask,
                'segment_ids': segment_ids
            }
            for _, input_ids, input_mask, segment_ids in choices_features
        ]
        self.label = label

    
def process_examples(data):
    """Given the examples in a pandas df format, process examples into example class"""
    examples = []
    labels = []
    questions = []
    anscands = []
    
    
    for index, row in data.iterrows(): 
        example = InputExample(
                    qid=row.id,
                    question=row.stem,
                    choice_0=str(row.choices[0]).replace("'",""),
                    choice_1=str(row.choices[1]).replace("'",""),
                    choice_2=str(row.choices[2]).replace("'",""),
                    choice_3=str(row.choices[3]).replace("'",""),
                    choice_4=str(row.choices[4]).replace("'",""),
                    label=lab_order[row.answerKey]
                )
        examples.append(example)
        
    return examples 

def convert_examples_to_features(examples, tokenizer, max_seq_length, is_training):
    # For each quesiton, we generate five inputs: one for each answer choice. 
    
    # - [CLS] question [SEP] choice_1 [SEP]
    # - [CLS] question [SEP] choice_2 [SEP]
    # - [CLS] question [SEP] choice_3 [SEP]
    # - [CLS] question [SEP] choice_4 [SEP]
    # - [CLS] question [SEP] choice_5 [SEP]
    
    features = []
    # Loop through questions
    for example_index, example in enumerate(examples):
        question_tokens = tokenizer.tokenize(example.question)

        choices_features = []
        # For each question, loop through all answer choices 
        for choice_index, choice in enumerate(example.choices):
            # We create a copy of the question tokens in order to be
            # able to shrink it according to choice_tokens
            question_tokens_choice = question_tokens[:]
            choice_tokens = tokenizer.tokenize(choice)
            # Modifies `question_tokens_choice` and `choice_tokens` in
            # place so that the total length is less than the
            # specified length.  Account for [CLS], [SEP], [SEP] with
            # "- 3"
            _truncate_seq_pair(question_tokens_choice, choice_tokens, max_seq_length - 3)

            tokens = ["[CLS]"] + question_tokens_choice + ["[SEP]"] + choice_tokens + ["[SEP]"]
            segment_ids = [0] * (len(question_tokens_choice) + 2) + [1] * (len(choice_tokens) + 1)

            input_ids = tokenizer.convert_tokens_to_ids(tokens)
            input_mask = [1] * len(input_ids)

            # Zero-pad up to the sequence length.
            padding = [0] * (max_seq_length - len(input_ids))
            input_ids += padding
            input_mask += padding
            segment_ids += padding

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            choices_features.append((tokens, input_ids, input_mask, segment_ids))

        label = example.label

        features.append(
            InputFeatures(
                example_id = example.qid,
                choices_features = choices_features,
                label = label
            )
        )

    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()
            
def select_field(features, field):
    """Yields a list, length equal to the total number of examples,
    where each item is a list of arrays,
    each array representing the feature array"""
    return [
        [
            choice[field]   # Grab the feature array of that choice.
            for choice in feature.choices_features  # Loop through 5 choices of that example
        ]
        for feature in features   # loop through each example
    ]



In [6]:
# Process inputs 

train_examples= process_examples(train.iloc[0:10])
train_features = convert_examples_to_features(
                    examples=train_examples, 
                    tokenizer=tokenizer, 
                    max_seq_length=50, 
                    is_training=True)

dev_examples= process_examples(dev.iloc[0:10])
dev_features = convert_examples_to_features(
                    examples=dev_examples, 
                    tokenizer=tokenizer, 
                    max_seq_length=50, 
                    is_training=True)


In [7]:
def create_inputs_from_features(features):
    input_ids = torch.tensor(select_field(train_features, 'input_ids'), dtype=torch.long)
    input_mask = torch.tensor(select_field(train_features, 'input_mask'), dtype=torch.long)
    segment_ids = torch.tensor(select_field(train_features, 'segment_ids'), dtype=torch.long)
    label = torch.tensor([f.label for f in features], dtype=torch.long)
    
    return input_ids, input_mask, segment_ids, label

all_input_ids, all_input_mask, all_segment_ids, all_label = create_inputs_from_features(train_features)
train_data = [all_input_ids, all_input_mask, all_segment_ids, all_label]


In [8]:
dev_data = create_inputs_from_features(dev_features)

In [9]:
print(all_input_ids.shape)
print(all_input_mask.shape)
print(all_segment_ids.shape)
print(all_label.shape)


torch.Size([10, 5, 50])
torch.Size([10, 5, 50])
torch.Size([10, 5, 50])
torch.Size([10])


In [10]:
from transformers import BertForMultipleChoice

In [11]:
model = BertForMultipleChoice.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

In [13]:
# # The forward method is the actual code that runs during the forward pass (like the predict method in sklearn or keras).

output = model.forward(
        input_ids=all_input_ids,
        attention_mask=all_input_mask,
        token_type_ids=all_segment_ids,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=all_label,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=True,
    )

In [14]:
type(output)

transformers.modeling_outputs.MultipleChoiceModelOutput

Output is of the class `MultipleChoiceModelOutput`. It contains the following elements:

            loss=loss,
            logits=reshaped_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        


In [15]:
output.keys()

odict_keys(['loss', 'logits'])

Looks like I didn't opt for it to return hidden states or attention. So I only got losses and logits. More info on what these mean [here - transformers.BertModel.forward](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertModel.forward)

`output['logits'][0]`



In [24]:
output['logits'][0]

tensor([-0.9159, -0.9222, -0.9232, -0.9734, -0.9024], grad_fn=<SelectBackward>)

In [40]:
torch.nn.functional.softmax(output['logits'][0])

tensor([0.2023, 0.2010, 0.2008, 0.1910, 0.2050], grad_fn=<SoftmaxBackward>)

In [44]:
# So here are the guesses!
for i in range(0, 10):
    predicted = torch.argmax(torch.nn.functional.softmax(output['logits'][i]))
    actual = int(all_label[i])
    print("predicted:", predicted,
          "actual:", actual, 
          predicted==actual)

predicted: tensor(4) actual: 0 tensor(False)
predicted: tensor(4) actual: 1 tensor(False)
predicted: tensor(0) actual: 0 tensor(True)
predicted: tensor(1) actual: 3 tensor(False)
predicted: tensor(0) actual: 2 tensor(False)
predicted: tensor(1) actual: 3 tensor(False)
predicted: tensor(4) actual: 4 tensor(True)
predicted: tensor(4) actual: 1 tensor(False)
predicted: tensor(2) actual: 4 tensor(False)
predicted: tensor(3) actual: 3 tensor(True)


Let's see if we can train it some more.

In [ ]:
predictions = model.predict()

location="models/predictions/BertForMultipleChoice_{}_dev_predictions".format(runtype)
pickle_out = open(location, "wb")
pickle.dump(predictions, pickle_out)
pickle_out.close()

In [ ]:
location='models/{NAME}.h5'.format(NAME=NAME)

model.save(location)